In [1]:
import torch
import pandas as pd
import numpy as np
from matplotlib import pyplot  as plt
import seaborn as sns
from IPython.core.display import display, HTML

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_372652/1482311168.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
print(torch.__version__)
print(torch.cuda.is_available())

1.14.0a0+44dac51
True


# Study Dataset

In [ ]:
train_set = pd.read_csv('./LungDiseaseDataset/train.csv')
train_set.rename(columns={'Pleural Effusion':'Effusion'}, inplace=True)
display('original train_set',train_set.head())
print()

val_set = pd.read_csv('./LungDiseaseDataset/valid.csv')
val_set.rename(columns={'Pleural Effusion':'Effusion'}, inplace=True)
display('original val_set', val_set.head())
print()

bowen_dataset = pd.read_csv('./LungDiseaseDataset/CheXpert_dataset.csv')
display('bowen_full_set', bowen_dataset.head())
print()


In [ ]:
# Display bowen dataset distribution
display('Bowen Class Distribution', bowen_dataset.groupby('labels').count())
print()

# Display original dataset distribution
bowen_classes = list(bowen_dataset.groupby('labels').groups.keys())
display('Original ChesXpert Class Distribution', train_set.loc[:,bowen_classes].count())

In [ ]:
# Training Set Distribution
all_classes = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

train_set_disease = train_set.loc[:,all_classes].fillna(-2).astype(np.int8)

temp1 = (train_set_disease==1).sum(axis=1)
sns.countplot(data=temp1.to_frame(), x=0)
plt.title("Train Set")
plt.xlabel("Number for Simultaneous Observation Classes in the Same Patient Record")
plt.ylabel('Number of Patients')
plt.tick_params(axis='x', labelrotation=0)
plt.show()


m1=(train_set_disease==1).sum(axis=0)
m0=(train_set_disease==0).sum(axis=0)
mu=(train_set_disease==-1).sum(axis=0)
mna=(train_set_disease==-2).sum(axis=0)
train_set_disease_summary = pd.DataFrame([m1, m0, mu, mna]).transpose()
train_set_disease_summary.columns=['Positive','Negative', 'Uncertain', 'No Mention']
train_set_disease_summary.sort_values(["Positive","Negative"]).plot(kind='bar', figsize=(20,10), stacked=True, rot=45)
#train_set_disease_summary.iloc[:,:3].plot(kind='bar', figsize=(20,10))
plt.tick_params('x', labelrotation=45)
plt.title("Train Set")
plt.show()

display(train_set_disease_summary)

In [ ]:
# Val Set Distribution

val_set_disease = val_set.loc[:,all_classes].fillna(-2).astype(np.int8)

temp_acc = 0
temp = (val_set.loc[:,all_classes].sum(axis=1)>1.5).sum()/val_set.shape[0]
print("In validation set, {}% are multilabel patient.".format(temp*100))
temp_acc=temp

temp = (val_set.loc[:,all_classes].sum(axis=1)<0.1).sum()/val_set.shape[0]
print("In validation set, {}% are multilabel patient.".format(temp*100))
temp_acc = temp_acc + temp

temp = (val_set.loc[:,all_classes].sum(axis=1)==1.0).sum()/val_set.shape[0]
print("In validation set, {}% are multilabel patient.".format(temp*100))
temp_acc = temp_acc + temp

assert int(temp_acc*100) == int(100)

temp = val_set_disease.loc[:,all_classes].sum(axis=1)
sns.countplot(temp.to_frame(), x=0)
plt.xlabel("Number for Simultaneous Observation Classes in the Same Patient Record")
plt.ylabel('Number of Patients')
plt.tick_params(axis='x', labelrotation=0)
plt.title("Val Set")
plt.show()

# display(val_set_disease.sum().sort_values())
# ax = val_set_disease.sum().sort_values().plot(kind='bar',rot=45, figsize=(20,10), title="Multilabel Observation Class Distribution in Validation Set")
# ax.set_ylabel('Count')
# plt.show()

temp1=(val_set.loc[:,all_classes]==1).sum().sort_values()
temp2=(val_set.loc[:,all_classes]==0).sum()
temp=pd.DataFrame({"1":temp1, "0":temp2}).sort_values("1")
display(temp)
temp.plot(kind='bar', rot=45, figsize=(20,10), stacked=True, title="Multilabel Observation Class Distribution in Validation Set")

# plt.figure(figsize=(20,10))
# aa = temp.reset_index()
# aa = aa.melt(id_vars='index')
# axs = sns.barplot(data=aa, x='index', y='value', hue='variable')
# axs.tick_params(axis='x', labelrotation=45)

In [ ]:
# Inspect co-occurance
train_set_disease_temp = train_set.loc[:,all_classes]
plt.figure(figsize=(20,10))
sns.heatmap(train_set_disease_temp.corr(numeric_only=False), annot=True, fmt=".2f")
plt.title('Train Set Correlation Matrix Excluding NaN element (White color Heatmap entry has "NaN" value, because at least one involved column has non-varying value).')
plt.show()

val_set_disease_temp = val_set.loc[:,all_classes]
plt.figure(figsize=(20,10))
sns.heatmap(val_set_disease_temp.corr(numeric_only=False), annot=True, fmt=".2f")
plt.title('Val Set Correlation Matrix Excluding NaN element (White color Heatmap entry has "NaN" value, because at least one involved column has non-varying value).')
plt.show()

# Create DataLoader

In [3]:
import os, pickle
from PIL.ImageFile import Image

from torch import as_tensor, int64, concat, cat
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T

In [4]:
df = pd.read_csv('./LungDiseaseDataset/train.csv')
df.head()
df.keys()

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')

In [5]:
class CustomImageDataset(Dataset):

    def __init__(self, data_root_path, csv_path, disease_classes, transform=None, target_transform=None, data_root_directory=None):
        self.img_labels = []
        self.img_paths = []
        
        data = pd.read_csv(os.path.join(csv_path))
        data_path = data.loc[:, 'Path'].to_numpy()
        data_target = data.loc[:,disease_classes].to_numpy()

        for path, target in zip(data_path, data_target):
            self.img_labels.append(target)
            self.img_paths.append(os.path.join(data_root_path, path))


        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):     
        image = self.img_paths[idx]
        image = Image.open(self.img_paths[idx])
        label = self.img_labels[idx]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
            
        #label = as_tensor([label], dtype=int64)

        return image, label

csv_path = "./LungDiseaseDataset/CheXpert-v1.0-small/train.csv"
disease_classes = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']


dataset = CustomImageDataset("/home/boon1987/Desktop/temp/lung_disease/LungDiseaseDataset", csv_path, disease_classes)


# Dataloader from Robus Deep AUC

In [6]:
import numpy as np
import torch 
from torch.utils.data import Dataset
import torchvision.transforms as tfs
import cv2
from PIL import Image
import pandas as pd
import pdb

class CheXpert(Dataset):
    '''
    Reference: 
        @inproceedings{yuan2021robust,
            title={Large-scale Robust Deep AUC Maximization: A New Surrogate Loss and Empirical Studies on Medical Image Classification},
            author={Yuan, Zhuoning and Yan, Yan and Sonka, Milan and Yang, Tianbao},
            booktitle={Proceedings of the IEEE/CVF International Conference on Computer Vision},
            year={2021}
            }
    '''
    def __init__(self, 
                 csv_path, 
                 image_root_path='',
                 image_size=320,
                 class_index=0, 
                 use_frontal=True,
                 use_upsampling=True,
                 flip_label=False,
                 shuffle=True,
                 seed=123,
                 verbose=True,
                 upsampling_cols=['Cardiomegaly', 'Consolidation'],
                 train_cols=['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis',  'Pleural Effusion'],
                 mode='train'):
        
    
        # load data from csv
        self.df = pd.read_csv(csv_path)
        self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0-small/', '')
        self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0/', '')
        if use_frontal:
            self.df = self.df[self.df['Frontal/Lateral'] == 'Frontal']  
            
        # upsample selected cols
        if use_upsampling:
            assert isinstance(upsampling_cols, list), 'Input should be list!'
            sampled_df_list = []
            for col in upsampling_cols:
                print ('Upsampling %s...'%col)
                sampled_df_list.append(self.df[self.df[col] == 1])
            self.df = pd.concat([self.df] + sampled_df_list, axis=0)


        # impute missing values 
        for col in train_cols:
            if col in ['Edema', 'Atelectasis']:
                self.df[col].replace(-1, 1, inplace=True)        # Set uncertain observation disease to one (positive)
                self.df[col].fillna(0, inplace=True)             # Set non-mention observation disease to zero (negative)
            elif col in ['Cardiomegaly','Consolidation',  'Pleural Effusion']:
                self.df[col].replace(-1, 0, inplace=True)        # Set uncertain observation disease to zero (negative)
                self.df[col].fillna(0, inplace=True)             # Set non-mention observation disease to zero (negative)
            else:
                self.df[col].fillna(0, inplace=True)             # Set non-mention observation disease to zero (negative)
        
        self._num_images = len(self.df)
        
        # 0 --> -1
        if flip_label and class_index != -1: # In multi-class mode we disable this option!
            self.df.replace(0, -1, inplace=True)                 # In multiclass mode, randomly flip the negative to uncertain
            
        # shuffle data rows
        if shuffle:
            data_index = list(range(self._num_images))
            np.random.seed(seed)
            np.random.shuffle(data_index)
            self.df = self.df.iloc[data_index]                    
        
        
        assert class_index in [-1, 0, 1, 2, 3, 4], 'Out of selection!'
        assert image_root_path != '', 'You need to pass the correct location for the dataset!'

        if class_index == -1: # 5 classes
            print ('Multi-label mode: True, Number of classes: [%d]'%len(train_cols))
            self.select_cols = train_cols
            self.value_counts_dict = {}
            for class_key, select_col in enumerate(train_cols):
                class_value_counts_dict = self.df[select_col].value_counts().to_dict()
                self.value_counts_dict[class_key] = class_value_counts_dict
        else:       # 1 class
            self.select_cols = [train_cols[class_index]]  # this var determines the number of classes
            self.value_counts_dict = self.df[self.select_cols[0]].value_counts().to_dict()
        
        self.mode = mode
        self.class_index = class_index
        self.image_size = image_size
        
        self._images_list =  [image_root_path+path for path in self.df['Path'].tolist()]
        if class_index != -1:
            self._labels_list = self.df[train_cols].values[:, class_index].tolist()
        else:
            self._labels_list = self.df[train_cols].values.tolist()
    
        if verbose:
            if class_index != -1:
                print ('-'*30)
                if flip_label:
                    self.imratio = self.value_counts_dict[1]/(self.value_counts_dict[-1]+self.value_counts_dict[1])
                    print('Found %s images in total, %s positive images, %s negative images'%(self._num_images, self.value_counts_dict[1], self.value_counts_dict[-1] ))
                    print ('%s(C%s): imbalance ratio is %.4f'%(self.select_cols[0], class_index, self.imratio ))
                else:
                    self.imratio = self.value_counts_dict[1]/(self.value_counts_dict[0]+self.value_counts_dict[1])
                    print('Found %s images in total, %s positive images, %s negative images'%(self._num_images, self.value_counts_dict[1], self.value_counts_dict[0] ))
                    print ('%s(C%s): imbalance ratio is %.4f'%(self.select_cols[0], class_index, self.imratio ))
                print ('-'*30)
            else:
                print ('-'*30)
                imratio_list = []
                for class_key, select_col in enumerate(train_cols):
                    imratio = self.value_counts_dict[class_key][1]/(self.value_counts_dict[class_key][0]+self.value_counts_dict[class_key][1])
                    imratio_list.append(imratio)
                    print('Found %s images in total, %s positive images, %s negative images'%(self._num_images, self.value_counts_dict[class_key][1], self.value_counts_dict[class_key][0] ))
                    print ('%s(C%s): imbalance ratio is %.4f'%(select_col, class_key, imratio ))
                    print ()
                self.imratio = np.mean(imratio_list)
                self.imratio_list = imratio_list
                print ('-'*30)
            
    @property        
    def class_counts(self):
        return self.value_counts_dict
    
    @property
    def imbalance_ratio(self):
        return self.imratio

    @property
    def num_classes(self):
        return len(self.select_cols)
       
    @property  
    def data_size(self):
        return self._num_images 
    
    def image_augmentation(self, image):
        img_aug = tfs.Compose([tfs.RandomAffine(degrees=(-15, 15), translate=(0.05, 0.05), scale=(0.95, 1.05), fill=128)]) # pytorch 3.7: fillcolor --> fill
        image = img_aug(image)
        return image
    
    def __len__(self):
        return self._num_images
    
    def __getitem__(self, idx):

        image = cv2.imread(self._images_list[idx], 0)
        image = Image.fromarray(image)
        if self.mode == 'train':
            image = self.image_augmentation(image)
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        
        # resize and normalize; e.g., ToTensor()
        image = cv2.resize(image, dsize=(self.image_size, self.image_size), interpolation=cv2.INTER_LINEAR)  
        image = image/255.0
        __mean__ = np.array([[[0.485, 0.456, 0.406]]])
        __std__ =  np.array([[[0.229, 0.224, 0.225]  ]]) 
        image = (image-__mean__)/__std__
        image = image.transpose((2, 0, 1)).astype(np.float32)
        if self.class_index != -1: # multi-class mode
            label = np.array(self._labels_list[idx]).reshape(-1).astype(np.float32)
        else:
            label = np.array(self._labels_list[idx]).reshape(-1).astype(np.float32)
        return image, label

In [7]:
root = "/home/boon1987/Desktop/temp/lung_disease/LungDiseaseDataset/"
csv_path = "/home/boon1987/Desktop/temp/lung_disease/LungDiseaseDataset/CheXpert-v1.0-small/"
root = '/home/boon1987/Desktop/temp/lung_disease/LungDiseaseDataset/CheXpert-v1.0-small/'
train_cols=['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis',  'Pleural Effusion']

traindSet = CheXpert(csv_path=root+'train.csv', image_root_path=root, use_upsampling=True, use_frontal=True, image_size=320, mode='train', class_index=-1)
testSet =  CheXpert(csv_path=root+'valid.csv',  image_root_path=root, use_upsampling=False, use_frontal=True, image_size=320, mode='valid', class_index=-1)
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=32, num_workers=2, drop_last=True, shuffle=True)
testloader =  torch.utils.data.DataLoader(testSet, batch_size=32, num_workers=2, drop_last=False, shuffle=False)

/tmp/ipykernel_372652/2439040562.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0-small/', '')
/tmp/ipykernel_372652/2439040562.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0/', '')


Upsampling Cardiomegaly...
Upsampling Consolidation...
Multi-label mode: True, Number of classes: [5]
------------------------------
Found 227395 images in total, 48021 positive images, 179374 negative images
Cardiomegaly(C0): imbalance ratio is 0.2112

Found 227395 images in total, 77866 positive images, 149529 negative images
Edema(C1): imbalance ratio is 0.3424

Found 227395 images in total, 27217 positive images, 200178 negative images
Consolidation(C2): imbalance ratio is 0.1197

Found 227395 images in total, 70593 positive images, 156802 negative images
Atelectasis(C3): imbalance ratio is 0.3104

Found 227395 images in total, 94036 positive images, 133359 negative images
Pleural Effusion(C4): imbalance ratio is 0.4135

------------------------------
Multi-label mode: True, Number of classes: [5]
------------------------------
Found 202 images in total, 66 positive images, 136 negative images
Cardiomegaly(C0): imbalance ratio is 0.3267

Found 202 images in total, 42 positive image

/tmp/ipykernel_372652/2439040562.py:38: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0-small/', '')
/tmp/ipykernel_372652/2439040562.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0/', '')


In [9]:
# Compute balanced class weight for multilabel classification
from sklearn.utils.class_weight import compute_class_weight

temp_targets = np.array(traindSet.df.loc[:, traindSet.select_cols]).astype(dtype=np.int64)

class_weights=[]
for i in np.arange(temp_targets.shape[1]):
    temp_y = temp_targets[:,i]
    class_weights.append(list(compute_class_weight('balanced', np.array([0,1]), y=temp_y)))
class_weights = np.array(class_weights)
display(class_weights)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


array([[0.63385719, 2.36766206],
       [0.7603709 , 1.46016875],
       [0.567982  , 4.17744424],
       [0.72510236, 1.61060587],
       [0.85256713, 1.20908482]])

# Create Model

In [10]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights, densenet121, DenseNet121_Weights

img = read_image("./LungDiseaseDataset/CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg")
img = np.repeat(img, repeats=3, axis=0)

# Step 1: Initialize model with the best available weights
weights = DenseNet121_Weights.DEFAULT
model = densenet121(weights=weights)


In [11]:
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

isopod: 28.5%


In [12]:
from libauc.models import densenet121

In [13]:
model = densenet121(pretrained=False)
model.eval()
model.cuda()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (elu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (elu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (elu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (elu

In [14]:
img = read_image("./LungDiseaseDataset/CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg")
img = np.repeat(img, axis=0, repeats=0)

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

batch = batch.cuda()
model(batch)

RuntimeError: Non-empty 4D data tensor expected but got a tensor with sizes [1, 0, 320, 389]

In [ ]:
# paramaters
SEED = 123
BATCH_SIZE = 32
 
lr = 0.1 # using smaller learning rate is better
gamma = 500
imratio = traindSet.imratio_list 
weight_decay = 1e-5
margin = 1.0

# model
model = densenet121(pretrained=True, last_activation=None, activations='relu', num_classes=5)
model = model.cuda()

In [ ]:
for x, y in trainloader:
    break

In [ ]:
image = cv2.imread('/home/boon1987/Desktop/temp/lung_disease/LungDiseaseDataset/CheXpert-v1.0-small/train/patient17953/study5/view1_frontal.jpg', 0)
image = Image.fromarray(image)
image = np.array(image)
image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

In [ ]:
t = torch.tensor([[1, 2,3], [4,5,6],[7,8,9],[10,11,12]]).float()
display(t.shape)
display(t)
index = torch.tensor([[0,0],[2,3],[1,2]])
display(index, index.shape)
display( torch.gather(t, 0, index) )
#torch.gather(t, 1, torch.tensor([[0, 0], [1, 0]]))

In [ ]:
t = torch.tensor([ [[1, 2,3], [4,5,6],[7,8,9],[10,11,12]], [[13, 14,15], [16,17,18],[7,8,9],[10,11,12]] ]).float()
t.shape
t.data.view((-1,2,4))

In [ ]:
import torch

# test without view
X = torch.tensor([[[0.25]],[[ 0.75]]],requires_grad=True,)
Y = torch.tensor([[[0.25]],[[ 0.75]]],requires_grad=True,)
y=(X*Y)**2
y.sum().backward()
print(f"X.grad: {X.grad, Y.grad}")
y=(X*Y)**2
y.sum().backward()
print(f"X.grad: {X.grad, Y.grad}")
# y=(X*Y)**2
# y.sum().backward()
# print(f"X.grad: {X.grad, Y.grad}")

# # print()

# test with view
X.grad.zero_()
Y.grad.zero_()
print(f"X.grad: {X.grad, Y.grad}")


X_view = X.view((2,1,1))
y=(X_view*Y)**2
y.sum().backward()
print(f"X.grad: {X.grad, Y.grad, X_view.grad}")
y=(X_view*Y)**2
y.sum().backward()
print(f"X.grad: {X.grad, Y.grad, X_view.grad}")


In [ ]:
loss = torch.nn.BCEWithLogitsLoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(input, target)
output.backward()


In [ ]:
import time

In [ ]:
a = time.time()

In [ ]:
b = time.time()

In [ ]:
b-a

# Create custom multilabel focal loss function

In [ ]:
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight

class Custom_MultiLabel_FocalLoss(torch.nn.Module): 
    """
    Focal Loss: Modified based on the libauc focal loss
    Reference: https://amaarora.github.io/2020/06/29/FocalLoss.html
    """
    def __init__(self, alpha=None, gamma=2, num_classes=5):
        super(Custom_MultiLabel_FocalLoss, self).__init__()
        if alpha is None:
            self.alpla = (torch.tensor([1.0]*5)).repeat(2, 1)
            self.alpha = self.alpla.cuda()
        else:
            self.alpha = torch.tensor(alpha).cuda()
        self.gamma = torch.tensor(gamma).cuda()

    def forward(self, inputs, targets):

            
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)


        # Modified for multilabel classification
        targets = targets.type(torch.long)
        at = self.alpha.gather(dim=0, index=targets)
        display(self.alpha)
        display(targets)
        display('at',at)
        display('pt',pt)
        display('BCE_loss', BCE_loss)
        F_loss = at.view(-1)*(1-pt.view(-1))**self.gamma * BCE_loss.view(-1)

        return F_loss.mean()


def calculate_multilabel_class_weight(targets):
    # Compute balanced class weight for multilabel classification

    class_weights=[]
    for i in np.arange(targets.shape[1]):
        temp_y = targets[:,i]
        class_weights.append(list(compute_class_weight('balanced', np.array([0,1]), y=temp_y)))
    class_weights = torch.as_tensor(np.array(class_weights).transpose())
    return class_weights

temp_targets = np.array(traindSet.df.loc[:, traindSet.select_cols]).astype(dtype=np.int64).copy()
class_weights = torch.as_tensor(calculate_multilabel_class_weight(temp_targets))
display(class_weights)

In [ ]:
loss_ln1 = Custom_MultiLabel_FocalLoss()
loss_ln2 = Custom_MultiLabel_FocalLoss(alpha=class_weights)
display(loss_ln1.alpha, loss_ln2.alpha)

for x, y in trainloader:
    break
ypred=torch.randint_like(y, high=2, dtype=torch.float64)
display(x.shape, y.shape, ypred.shape)

loss_ln1(ypred.cuda(), y.cuda())


In [ ]:
dir(torch.backends)

In [92]:
from torch import nn, optim
from torch.nn import functional as F

# Define model
class TheModelClass(nn.Module):

    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 3, 3, padding="same")
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = x.view(-1, 1)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()
output = model(torch.randn((1, 3, 20, 20)))
output.size()

model_traced1 = torch.jit.script(model, torch.randn(1,3,26,27))
model_traced2 = torch.jit.script(model, torch.randn(1,3,28,29))
x = torch.randn(1,3,25,25)
print(model_traced1(x).size())
print(model_traced2(x).size())
print(model_traced1(x)[0,0,0,:10])
print(model_traced2(x)[0,0,0,:10])
print(model_traced1(x)[0,0,0,:-10])
print(model_traced2(x)[0,0,0,:-10])

# # Initialize optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# # Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

torch.Size([1, 3, 25, 25])
torch.Size([1, 3, 25, 25])
tensor([0.4424, 0.2022, 0.5121, 0.3441, 0.4273, 0.0000, 0.0000, 0.4418, 0.0000,
        0.0000], grad_fn=<SliceBackward0>)
tensor([0.4424, 0.2022, 0.5121, 0.3441, 0.4273, 0.0000, 0.0000, 0.4418, 0.0000,
        0.0000], grad_fn=<SliceBackward0>)
tensor([0.4424, 0.2022, 0.5121, 0.3441, 0.4273, 0.0000, 0.0000, 0.4418, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1970, 0.2277, 0.4336],
       grad_fn=<SliceBackward0>)
tensor([0.4424, 0.2022, 0.5121, 0.3441, 0.4273, 0.0000, 0.0000, 0.4418, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1970, 0.2277, 0.4336],
       grad_fn=<SliceBackward0>)


/usr/local/lib/python3.8/dist-packages/torch/jit/_script.py:1243: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


In [1]:
import torch
torch.jit.save(model_traced1, './model_trace')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model_traced1' is not defined

In [6]:
model_traced_load = torch.jit.load('./model_trace')
print(model_traced_load.code)

def forward(self,
    x: Tensor) -> Tensor:
  conv1 = self.conv1
  x0 = __torch__.torch.nn.functional.relu((conv1).forward(x, ), False, )
  return x0



In [66]:

import torch, time

class MyModule(torch.nn.Module):

    def __init__(self):
        super(MyModule, self).__init__()
        self.linear = torch.nn.Linear(4, 4, bias=False)
    
    def forward(self, xs):
        h = torch.zeros(3, 4)
        for i in range(xs.size(0)):
            h = torch.tanh(h + self.linear(xs[i]))
        return h

mymod = MyModule()


In [71]:

x = torch.rand(10, 3, 4)
y = mymod(x)
y.sum().backward()

print([i.grad for i in mymod.parameters()])
mymod.zero_grad(set_to_none=True)
print([i.grad for i in mymod.parameters()])


[tensor([[8.6650, 7.0595, 8.5649, 4.8167],
        [0.8052, 0.6646, 0.4260, 0.3602],
        [9.2512, 7.7348, 9.0918, 5.0951],
        [1.6403, 1.0277, 1.0677, 0.8306]])]
[None]


In [ ]:

a=time.time()
for i in range(10000):
    y = mymod(x)
b=time.time()
print(b-a)


y.sum().backward()
mymod.zero_grad
mymod.zero_grad(set_to_none=True)
mymod = mymod.train()
traced_mymod = torch.jit.trace(mymod,x)
a=time.time()
for i in range(10000):
    y = traced_mymod(x)
b=time.time()
print(b-a)


In [53]:
mymod.zero_grad()

In [58]:
print([i for i in mymod.linear.parameters()])

[Parameter containing:
tensor([[-3.0542e-01,  2.7690e-01, -1.3966e-01, -3.3996e-01],
        [ 2.3003e-01,  4.1637e-01, -9.5947e-02, -7.6008e-02],
        [ 9.9899e-02, -3.4648e-04, -4.9217e-01,  4.8397e-01],
        [ 3.5403e-01,  3.6949e-02,  2.3214e-01, -1.8929e-02]],
       requires_grad=True)]
